# Core Concepts

## Cluster Architecture

### MasterNodes
- API Server
- etcd
- Scheduler
- Controller Manager

### Worker Node
- Kubelet
- Container Runtime Engine
- Kube-proxy

---

## Etcd

distributed key-value store, simple, reliable, fast

1. download binary
1. extract
1. run etcd
   ```
   ./etcd
   ```
   etcd listens on port **2379** by default
   
   default cli client `etcdctl`
   ```
   ./etcdctl set <ke1> <valu1>
   ./etcdctl get <ke1>|
   ```
---

## Etcd in K8s


### k8s setup

1. Manual
   etcd is run as a binary in master node, etcd listens on maspternodeip, port 2379
2. kubdeadm
   in a cluster set ups using kubeadm, etcd is run as a pod in kubesystem

In [5]:
kubectl get pods -n kube-system

NAME                                        READY   STATUS    RESTARTS   AGE
coredns-5c98db65d4-6n8rw                    1/1     Running   4          6d1h
coredns-5c98db65d4-wt29k                    1/1     Running   4          6d1h
etcd-minikube                               1/1     Running   2          6d1h
kube-addon-manager-minikube                 1/1     Running   2          6d1h
kube-apiserver-minikube                     1/1     Running   2          6d1h
kube-controller-manager-minikube            1/1     Running   2          6d1h
kube-proxy-cdsjx                            1/1     Running   2          6d1h
kube-scheduler-minikube                     1/1     Running   2          6d1h
kubernetes-dashboard-7b8ddcb5d6-strk2       1/1     Running   5          6d1h
nginx-ingress-controller-5d9cf9c69f-jv79n   1/1     Running   2          6d1h
storage-provisioner                         1/1     Running   5          6d1h


In [21]:
kubectl exec etcd-minikube -n kube-system  etcdctl get / --prefix -keys-only #fix this later

Error: unknown flag: --prefix


Examples:
  # Get output from running 'date' command from pod mypod, using the first container by default
  kubectl exec mypod date
  
  # Get output from running 'date' command in ruby-container from pod mypod
  kubectl exec mypod -c ruby-container date
  
  # Switch to raw terminal mode, sends stdin to 'bash' in ruby-container from pod mypod
  # and sends stdout/stderr from 'bash' back to the client
  kubectl exec mypod -c ruby-container -i -t -- bash -il
  
  # List contents of /usr from the first container of pod mypod and sort by modification time.
  # If the command you want to execute in the pod has any flags in common (e.g. -i),
  # you must use two dashes (--) to separate your command's flags/arguments.
  # Also note, do not surround your command and its flags/arguments with quotes
  # unless that is how you would execute it normally (i.e., do ls -t /usr, not "ls -t /usr").
  kubectl exec mypod -i -t -- ls -t /usr
  
  # Get output from running 

: 1